In [1]:
# Parameters
cpu = 8
group_name = "DG_Glut"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name
dar_dir = '/data/combined_DARs_redo'
abc_dir = '/data/celltype_age_RPM_files'
#_type = 'up_peaks'

In [5]:
atac_rmp = pd.read_csv(f"{abc_dir}/{ct}_RPM.txt", sep = '\t')
# add three coolumns chr, start and end based on ethe index 
atac_rmp['chr'] = atac_rmp.index.str.split(':').str[0]
atac_rmp['start'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[0].astype(int)
atac_rmp['end'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[1].astype(int)
# put the chr start and end in the first column
cols = atac_rmp.columns.tolist()
cols = cols[-3:] + cols[:-3]
atac_rmp = atac_rmp[cols]
# remove chrY, chrM and chrX
atac_rmp = atac_rmp[~atac_rmp['chr'].isin(['chrY', 'chrM', 'chrX'])]
atac_rmp


,chr,start,end,DG Glut:2mo,DG Glut:9mo,DG Glut:18mo
chr1:3012480-3012981,chr1,3012480,3012981,2.535191,2.240782,2.096444
chr1:3060761-3061262,chr1,3060761,3061262,1.626886,1.176787,1.150766
chr1:3113502-3114003,chr1,3113502,3114003,5.274336,4.421410,4.154910
chr1:3112919-3113420,chr1,3112919,3113420,2.165229,1.654269,1.648291
chr1:3120282-3120783,chr1,3120282,3120783,2.359696,2.229503,2.111636
...,...,...,...,...,...,...
chr9:124439324-124439825,chr9,124439324,124439825,0.853759,1.026399,0.892508
chr9:124439828-124440329,chr9,124439828,124440329,0.818186,1.041437,1.143170
chr9:124438702-124439203,chr9,124438702,124439203,0.450595,0.718103,0.767177
chr9:124441952-124442453,chr9,124441952,124442453,0.640320,0.797057,0.786167


In [6]:
# add rows in dar but not atac-rmp
dar_df = pd.read_csv(f'{home_dir}/ml_input/{ct}/{ct}.aDAR_gene.csv')
# make the index as chr:start-end
dar_df.index = dar_df['chr'] + ':' + dar_df['start'].astype(str) + '-' + dar_df['end'].astype(str)
dar_df['log2(18mo/2mo)'] = np.log2((dar_df['18mo'] + 1)/(dar_df['2mo'] + 1))

not_peak_dar =dar_df.loc[~dar_df.index.isin(atac_rmp.index)]
not_peak_dar = not_peak_dar[['chr','start','end','2mo','9mo','18mo']]
not_peak_dar.columns = atac_rmp.columns
atac_rmp = pd.concat([atac_rmp, not_peak_dar], axis = 0)
atac_rmp

,chr,start,end,DG Glut:2mo,DG Glut:9mo,DG Glut:18mo
chr1:3012480-3012981,chr1,3012480,3012981,2.535191,2.240782,2.096444
chr1:3060761-3061262,chr1,3060761,3061262,1.626886,1.176787,1.150766
chr1:3113502-3114003,chr1,3113502,3114003,5.274336,4.421410,4.154910
chr1:3112919-3113420,chr1,3112919,3113420,2.165229,1.654269,1.648291
chr1:3120282-3120783,chr1,3120282,3120783,2.359696,2.229503,2.111636
...,...,...,...,...,...,...
chrX:20950965-20951466,chrX,20950965,20951466,3.559703,2.925048,2.863622
chr3:26329942-26330443,chr3,26329942,26330443,3.900169,4.202433,3.657866
chr3:26329942-26330443,chr3,26329942,26330443,3.900169,4.202433,3.657866
chr5:18358756-18359257,chr5,18358756,18359257,2.684832,2.449049,2.737395


In [7]:
atac_rmp.to_csv(f'{ct}.peak.bed', sep = '\t', index = False, header = False)

# use bedtools intersect to find the genes that overlap with the DMRs
command = f"bedtools intersect -a {ct}.peak.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.ATAC_peak_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a DG_Glut.peak.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > DG_Glut.ATAC_peak_gene.bed', returncode=0)

In [8]:
peak_gene_df = pd.read_csv(f'{ct}.ATAC_peak_gene.bed', sep='\t', header=None)
peak_gene_df.columns = ['chrom','peak_start','peak_end','2mo','9mo','18mo','gene_chrom','gene_start','gene_end','gene_id','strand', 'gene_name','gene_type']
peak_gene_df.index = peak_gene_df['chrom'] + ':' + peak_gene_df['peak_start'].astype(str) + '-' + peak_gene_df['peak_end'].astype(str)
# if the row index in dar and values in 
peak_gene_df

,chrom,peak_start,peak_end,2mo,9mo,18mo,gene_chrom,gene_start,gene_end,gene_id,strand,gene_name,gene_type
chr1:3204691-3205192,chr1,3204691,3205192,1.145461,1.075275,0.968466,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3208870-3209371,chr1,3208870,3209371,0.645063,0.466203,0.398780,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3210193-3210694,chr1,3210193,3210694,2.222146,1.932487,1.435609,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3210788-3211289,chr1,3210788,3211289,0.315417,0.278218,0.235470,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3209692-3210193,chr1,3209692,3210193,0.640320,0.503800,0.383589,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr3:26329942-26330443,chr3,26329942,26330443,3.900169,4.202433,3.657866,chr3,25424215,26334460,ENSMUSG00000063887.9,-,Nlgn1,protein_coding
chr3:26329942-26330443,chr3,26329942,26330443,3.900169,4.202433,3.657866,chr3,26329150,26335377,ENSMUSG00000074664.3,+,A830092H15Rik,antisense
chr3:26329942-26330443,chr3,26329942,26330443,3.900169,4.202433,3.657866,chr3,25424215,26334460,ENSMUSG00000063887.9,-,Nlgn1,protein_coding
chr5:18358756-18359257,chr5,18358756,18359257,2.684832,2.449049,2.737395,chr5,18263135,18362413,ENSMUSG00000057614.5,-,Gnai1,protein_coding


In [9]:
peak_gene_df['DAR'] = peak_gene_df.index.map(dar_df['log2(18mo/2mo)'].to_dict())
# for values in DAR, if na == 0, if > 0 return 1, if < 0 return -1
peak_gene_df['DAR'] = peak_gene_df['DAR'].fillna(0)
peak_gene_df['DAR'] = np.sign(peak_gene_df['DAR'])
peak_gene_df.head()

,chrom,peak_start,peak_end,2mo,9mo,18mo,gene_chrom,gene_start,gene_end,gene_id,strand,gene_name,gene_type,DAR
chr1:3204691-3205192,chr1,3204691,3205192,1.145461,1.075275,0.968466,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3208870-3209371,chr1,3208870,3209371,0.645063,0.466203,0.398780,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3210193-3210694,chr1,3210193,3210694,2.222146,1.932487,1.435609,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3210788-3211289,chr1,3210788,3211289,0.315417,0.278218,0.235470,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3209692-3210193,chr1,3209692,3210193,0.640320,0.503800,0.383589,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0


In [10]:
# drop dmr_id, gen_id, gene_strand, old-young
peak_gene_df['distance'] = np.minimum(abs(peak_gene_df['peak_start'] - peak_gene_df['gene_start']), abs(peak_gene_df['peak_start'] - peak_gene_df['gene_end']))
peak_gene_df = peak_gene_df.drop(columns=['chrom','peak_start','peak_end','gene_chrom','gene_id','strand','gene_type'])
peak_gene_df.head()

,2mo,9mo,18mo,gene_start,gene_end,gene_name,DAR,distance
chr1:3204691-3205192,1.145461,1.075275,0.968466,3203901,3673498,Xkr4,0.0,790
chr1:3208870-3209371,0.645063,0.466203,0.398780,3203901,3673498,Xkr4,0.0,4969
chr1:3210193-3210694,2.222146,1.932487,1.435609,3203901,3673498,Xkr4,0.0,6292
chr1:3210788-3211289,0.315417,0.278218,0.235470,3203901,3673498,Xkr4,0.0,6887
chr1:3209692-3210193,0.640320,0.503800,0.383589,3203901,3673498,Xkr4,0.0,5791


In [11]:
peak_gene_df.to_csv(f"{home_dir}/ml_input/{ct}/{ct}.peak_gene.csv", index = False)

In [12]:
subprocess.run(f"rm {ct}.peak.bed", shell=True)
subprocess.run(f"rm {ct}.ATAC_peak_gene.bed", shell=True)

CompletedProcess(args='rm DG_Glut.ATAC_peak_gene.bed', returncode=0)

## enhancer

In [13]:
atac_abc_dir = '/data/female-amb/ABC.Enhancer/ABC.By.ATAC.RPM.Raw.Cool'

In [14]:
mouse_genes = pd.read_csv(f"/home/qzeng_salk_edu/project/240321_abc_score/use.mouse_genes.csv", index_col =0)

In [15]:
def load_abc_df(ct, age):
    df = pd.read_csv(f'{atac_abc_dir}/{ct}/filtered.{ct}.{age}.abc_score.csv', index_col =0)
    if age == '8wk':
        age = '2mo'
    df.columns =  [f'{age}.activity',f'{age}.contact',f'{age}.abc_score']
    return df

In [16]:
young_abc_df = load_abc_df(ct, '8wk')
middle_abc_df = load_abc_df(ct, '9mo')
old_abc_df = load_abc_df(ct, '18mo')

In [17]:
# concat the three age groups
abc_df = pd.concat([young_abc_df, middle_abc_df, old_abc_df], axis=1)
#abc_df = abc_df.dropna()#
#abc_df = abc_df.fillna(0)

In [18]:
abc_df['chr'] = abc_df.index.map(lambda x: x.split(':')[0])
abc_df['start'] = abc_df.index.map(lambda x: int(x.split(':')[1].split('-')[0]))
abc_df['end'] = abc_df.index.map(lambda x: int(x.split(':')[1].split('-')[1]))
abc_df['gene_id'] = abc_df.index.map(lambda x: x.split(':')[1].split('-')[2])
abc_df['gene_name'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'gene_name'])

abc_df['gene_start'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'start'])
abc_df['gene_end'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'end'])
# add eg_distance, based on the start is more close to gene_start ot gene_end; use the smaller absolute value
#abc_df drop chr, start, end, gene_start, gene_end
#abc_df = abc_df.drop(columns=['chr','start','end','gene_start','gene_end'])
abc_df


,2mo.activity,2mo.contact,2mo.abc_score,9mo.activity,9mo.contact,9mo.abc_score,18mo.activity,18mo.contact,18mo.abc_score,chr,start,end,gene_id,gene_name,gene_start,gene_end
chr1:3360710-3361211-ENSMUSG00000051951,4.313857,16.0,0.025424,NaN,NaN,NaN,NaN,NaN,NaN,chr1,3360710,3361211,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:3407326-3407827-ENSMUSG00000051951,9.692541,13.0,0.046413,7.948010,10.0,0.033619,8.492119,13.0,0.034804,chr1,3407326,3407827,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:3671537-3672038-ENSMUSG00000051951,23.559017,8.0,0.069423,20.276073,12.0,0.102919,20.539079,12.0,0.077702,chr1,3671537,3672038,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:3670856-3671357-ENSMUSG00000051951,10.579502,8.0,0.031175,9.752667,12.0,0.049504,8.495917,12.0,0.032141,chr1,3670856,3671357,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:3672169-3672670-ENSMUSG00000051951,6.827704,8.0,0.020120,6.297501,12.0,0.031965,5.586721,12.0,0.021135,chr1,3672169,3672670,ENSMUSG00000051951,Xkr4,3205901,3671498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX:168674307-168674808-ENSMUSG00000087159,NaN,NaN,NaN,NaN,NaN,NaN,4.124527,9.0,0.032355,chrX,168674307,168674808,ENSMUSG00000087159,Gm15246,169320622,169368190
chrX:168672909-168673410-ENSMUSG00000087159,NaN,NaN,NaN,NaN,NaN,NaN,2.825642,9.0,0.022166,chrX,168672909,168673410,ENSMUSG00000087159,Gm15246,169320622,169368190
chrX:169798879-169799380-ENSMUSG00000035299,NaN,NaN,NaN,NaN,NaN,NaN,1.769824,13.0,0.021562,chrX,169798879,169799380,ENSMUSG00000035299,Mid1,169685199,170005736
chrX:169940294-169940795-ENSMUSG00000035299,NaN,NaN,NaN,NaN,NaN,NaN,1.363448,20.0,0.025555,chrX,169940294,169940795,ENSMUSG00000035299,Mid1,169685199,170005736


In [19]:
abc_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.abc_enhancer.peak_gene.csv', index=None)